In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp cli

# CLI 
> Contains functions for calling Rough from the command line

In [ ]:
from fastcore.script import *
import numpy as np

from rough.data import *
from rough.profile import *
from rough.section import *
from pathlib import Path

In [ ]:
# | export
@call_parse
def rough(
    fname:str   = None,   #File name, path or directory with data files to be read
    ext:str     = '.txt', #Extension for the  files .txt or .csv
    result:str  = None,   #Directory to write results to, if None, writes to 'results' 
    
    level:bool  = True, #Perform plane levelling 
    form:bool   = True, #Remove form by polynomial subtraction
    deg:int     = 3,    #Degree of polynomial to remove
    smooth:bool = True, #Smooth the array by applying a gaussian
    sigma:int   = 1,    #Sigma for gaussian to be applied
    
    gen_rot:bool= True, #Generate rotational profiles and apply parameter calculation to them
    
    section:bool= True,     #Generate sections
    sec_how:str = 'square', #Type of section to generate, currently only supports 'square'
    sec_num:int = 100,      #Number of sections to generate
    
    params1D:list = [Ra,Rms,Rku,Rsk], # list of 1D parameters to calculate,
    params2D:list = [Sa,Sms,Sku,Ssk], #list of 2D parameters to calculate
):
    
    delims = {'.txt': None,
              '.csv': ','}
    
    path       = Path().cwd() if fname == None else Path(fname)
    
    #Figuring out where the results go
    if   path.is_dir():
        result_dir = path / 'results' if result == None else result
    elif path.is_file():
        result_dir = path.parent / 'results' if result == None else result # 
    

    if not path.exists(): 
        raise FileNotFoundError('Could not find file/directory check fname')
    if not result_dir.exists(): 
        result_dir.mkdir(Parents=True) #Make the results directory if it doesn't exist
        
    glob_pattern    = '*' + ext
    file_paths = [path] if path.is_file() else path.glob(glob_pattern)
    
    for file_path in file_paths:
        array = np.loadtxt(file_path,delimiter=delims[file_path.suffix])
        print(file_path)
        if level:
            array = plane_level(array)
            print('Got to level')
        if form:
            array = remove_form(array)
            print('Got to form')
        if smooth:
            array = smooth_image(array,sigma=sigma)
            print('got to smooth')
        if gen_rot:
            profiles = gen_rot_prof(array)
            print('got to profiles')
            #Helper function that computes parameters
        if section:
            sections = gen_sections(array, how=sec_how, number = sec_num)
            #Helper function that computes parameters
            section_results = None
            result_file_name = file_path.stem + '_section' + file_path.suffix #TODO add in .csv and other filer support
            result_path = result_dir / result_file_name 
            print(result_path)
            #np.savetxt(result_path, section_results, delim =delim) #Save the section results

In [ ]:
path = Path().cwd()/'example.txt'
np.loadtxt(path, delimiter=None)

array([[ 1.57419133e-04,  1.42266877e-04,  1.35691441e-04, ...,
        -7.36308144e-04, -7.41222638e-04, -7.37872231e-04],
       [ 1.59027040e-04,  1.44439161e-04,  1.40401461e-04, ...,
        -6.86596145e-04, -6.92912643e-04, -6.94630788e-04],
       [ 1.45829296e-04,  1.36737508e-04,  1.41029337e-04, ...,
        -6.06042695e-04, -6.12879493e-04, -6.18730593e-04],
       ...,
       [-1.24465439e-04, -1.02288970e-04, -6.34991852e-05, ...,
         2.75950565e-03,  2.81423047e-03,  2.85402902e-03],
       [-9.49129347e-05, -6.31949688e-05, -1.58049753e-05, ...,
         3.15482891e-03,  3.22523772e-03,  3.29882757e-03],
       [-5.87019947e-05, -2.16723500e-05,  3.29604742e-05, ...,
         3.50673568e-03,  3.63254488e-03,  3.77919674e-03]])

In [ ]:
path = Path()/'nonexistent_file.txt'
pattern = '*.txt'
path.exists()

False

In [ ]:
path = Path.cwd()
path

Path('E:/Archaeology/rough/rough')

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()